##### 导入库

In [37]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from collections import Counter
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

import sys
sys.path.append(r"G:\item\工作\特征算法12.1")
import SingleFlash_125_Features

##### 选择数据

In [38]:
n='★阴性整合.csv'
n='alln++.csv'
# p='★阳性整合.csv'
# p='纯冰毒-重复.csv'
p='纯海洛因-重复.csv'
# p="混吸.csv"
way="ice"

##### 读取数据

In [39]:
fea_dict={
    'all':[
"retang",
"fft_real_2",
"minindex",
"lumpiness",
"fft_angle_3",
"fft_imag_6",
"alt_stderr_min_50",
"erbc_num10_focus2",
"fft_imag_3",
"alt_rvalue_min_50"
],

 'ice':[
"fft_abs_22",
"lumpiness",
"fft_real_5",
"maxvalue",
"fft_imag_6",
"erbc_num10_focus2",
"fft_imag_3",
"maxindex"
],

 'her':[
"fft_real_2",
"fft_real_5",
"dmq",
"fft_angle_3",
"erbc_num10_focus2",
"fft_imag_3",
"alt_rvalue_min_50"
],

'mix':[
"minvalue",
"cv",
"lumpiness",
"alt_rvalue_min_50",
"fft_angle_3",
"alt_stderr_min_50",
"maxindex"
]
}

fea_list=fea_dict[way]
def read_data(tit,fea_list):
    path = 'E:\\desktop\\data\\'

    #读取数据
    try:
        data = pd.read_csv(path+tit,encoding='gbk')
    except:
        data = pd.read_csv(path+tit,encoding='utf-8')

    name=pd.DataFrame(list(data.iloc[:,0]),columns=["name"])

    try:
        # 对瞳孔半径列进行分割
        df=data['瞳孔半径'].astype('str').str.split(',',expand=True)
        # 修改分割后的字段名称
        df.columns=[str(i) for i in df.columns]
    except:
        df=pd.DataFrame(data.iloc[:,1:126])

    df=df.astype('float')

    fea_dict=dict(zip(fea_list,[[] for i in range(len(fea_list))]))


    for i in range(len(df)):
        row=list(df.iloc[i])
        ret=SingleFlash_125_Features.all_features(row)
        for fea in fea_list:
            fea_dict[fea].append(ret[fea])
    fea_df=pd.DataFrame(fea_dict)
    fea_df=shuffle(fea_df)
    return fea_df

n_fea=read_data(n,fea_list)
p_fea=read_data(p,fea_list)

n_fea["tag"]=[0 for i in range(len(n_fea))]
p_fea["tag"]=[1 for i in range(len(p_fea))]

print(len(n_fea),len(p_fea))


556 174


##### 过采样

In [40]:
#过采样
from imblearn.over_sampling import SVMSMOTE
svmsomte=SVMSMOTE(random_state=42)

np_fea=shuffle(pd.concat([n_fea,p_fea],axis=0))
x_data_=np_fea[fea_list]
y_data_=np_fea["tag"]

x_data,y_data=svmsomte.fit_resample(x_data_,y_data_)

#切分
x_train,x_test,y_train,y_test=train_test_split(x_data,y_data,test_size=0.3)
print(x_data.shape)

(1112, 9)


In [41]:
def jugde(y_pred,y_true):
    tn,fn,tp,fp=0,0,0,0
    for i in range(len(y_pred)):
        
        if y_pred[i]==y_true[i]:
            if y_true[i]:
                tp+=1
            else:
                tn+=1
        else:
            if y_true[i]:
                fp+=1
            else:
                fn+=1

    print("检出率： ",tp,"/",tp+fp,' ',round(tp*100/(tp+fp),2),"%")
    print("误判率： ",fn,"/",tn+fn,' ',round(fn*100/(tn+fn),2),"%")
    print("精确率： ",(tn+tp),"/",(tn+fp+tp+fp),' ',round((tn+tp)*100/(tn+fp+tp+fp),2),"%")


##### 搭建模型

In [42]:

#导入模型存储和加载方法
from joblib import dump,load

#n_estimators决策树数量 RandomForestClassifier
# rfc=RandomForestClassifier(n_estimators=400,max_depth=20,class_weight='balanced') #海洛因
rfc=RandomForestClassifier(n_estimators=700)
rfc.fit(x_train,y_train)
y_pred=rfc.predict(x_test)

print("测试集中：")
jugde(y_pred,list(y_test))

print("所有数据中：")
y_pred=rfc.predict(x_data_)
jugde(y_pred,list(y_data_))


#保存模型
dump(rfc,'rfc_'+way+".joblib")
print("保存成功")

测试集中：
检出率：  165 / 180   91.67 %
误判率：  15 / 154   9.74 %
精确率：  304 / 334   91.02 %
所有数据中：
检出率：  165 / 174   94.83 %
误判率：  15 / 556   2.7 %
精确率：  706 / 724   97.51 %
保存成功
